# Analysis of [Hui-Ling Yen]'s *A/California/04/2009* sequences and design of synonymously "barcoded" variants
This [iPython notebook](http://ipython.org/notebook.html) describes the analysis of pdmH1N1 influenza gene sequences and the design of synonymous "barcodes" which are intended to be selectively neutral but allow for distinguishing between variants by sequencing.

This analysis is performed by [Jesse Bloom](http://research.fhcrc.org/bloom/en.html) to contribute to a larger project being led by [Hui-Ling Yen].

## Compilation of a set of naturally occuring sequences
As a point of comparison, we assemble a set of naturally occurring sequences. To do this, on September-5-2015, I downloaded from the [Influenza Virus Resource](http://www.ncbi.nlm.nih.gov/genomes/FLU/FLU.html) all full-length human influenza coding DNA sequences for human H1N1 annotated as *pandemic H1N1*. Here are some details on that sequence set:

[Hui-Ling Yen]: http://sph.hku.hk/en/about-us/faculty-and-staff/academic-staff/yen,-hui-ling

In [1]:
seqdir = './sequences' 
allseqs = '%s/all_pdmH1N1_seqs.fasta' % seqdir
cdnas = ['PB2', 'PB1', 'PA', 'HA', 'NP', 'NA', 'M1', 'M2', 'NS1', 'NS2'] # we analyze these cDNAs
ignored_cdnas = ['PA-X', 'PB1-F2'] # we ignore these cDNAs

import Bio.SeqIO
seqs = [seq for seq in Bio.SeqIO.parse(allseqs, 'fasta')]
print('Read a total of %d sequences from %s' % (len(seqs), allseqs))
seqs_by_cdna = dict([(cdna, []) for cdna in cdnas])
for seq in seqs:
    cdna = seq.description.strip().split()[-1]
    if cdna in seqs_by_cdna:
        seqs_by_cdna[cdna].append(seq)
    elif cdna not in ignored_cdnas:
        print("Unrecognized cDNA of %s in sequence header %s" % (cdna, seq.description))
for cdna in cdnas:
    print("For %s, found %d sequences" % (cdna, len(seqs_by_cdna[cdna])))

Read a total of 68816 sequences from ./sequences/all_pdmH1N1_seqs.fasta
For PB2, found 5114 sequences
For PB1, found 5125 sequences
For PA, found 5358 sequences
For HA, found 11321 sequences
For NP, found 5457 sequences
For NA, found 8990 sequences
For M1, found 7608 sequences
For M2, found 7051 sequences
For NS1, found 5907 sequences
For NS2, found 5537 sequences


## Reference sequences for experimental work from [Hui-Ling Yen]'s plasmids
Our experiments will utilize pdmH1N1 sequences cloned into reverse genetics plasmids by [Hui-Ling Yen].

She has provided sequences for reverse-genetics plasmids encoding the genes for the strains *A/California/04/2009* and *A/Hong Kong/415742/2009*, which are both pdmH1N1. It appears that the references for these plasmids are [Yen, PNAS, 108:14264](http://www.pnas.org/content/108/34/14264.full) and [Wong, J Virology, 86:10558](http://jvi.asm.org/content/86/19/10558.abstract), respectively. From those references, it is clear that both viruses can be rescued from the reverse-genetics plasmids. According to [Hui-Ling Yen]'s e-mail, the coding sequences for the two viruses are identical for *PB1*, *NA*, *M*, and *NS* -- but differ for *PB2*, *PA*, *HA*, and *NP*. She also noted that there are four non-synonymous mutations in *PB1* (perhaps relative to the Genbank reference, I wasn't clear on this point?) that should be examined. Her rationale for preferring to use the *A/California/04/2009* virus over the *A/Hong Kong/415742/2009* is that the former is a more commonly used reference strain for studies, whereas the latter is less used and has some differences relative to many other pdmH1N1 strains. I had heard anecdotally that some *A/California/04/2009* strains are difficult to rescue and/or propagate in tissue-culture, but based on her prior work with this virus it doesn't seem to be a big problem for her particular variant.

In the analysis that follows, I will **only be examining the coding sequences, not the noncoding regions.** The reason is that the noncoding regions in Genbank sequences are often unreliable, and I'm going to assume that the noncoding regions in [Hui-Ling Yen]'s plasmids are fine since she has successfully used them in the past.

First, we read in the sequences that [Hui-Ling Yen] provided for the inserts in her reverse-genetics plasmids:

[Hui-Ling Yen]: http://sph.hku.hk/en/about-us/faculty-and-staff/academic-staff/yen,-hui-ling

In [15]:
plasmidseqs = '%s/plasmid_sequences_20150904.txt' % seqdir
plasmids = [seq for seq in Bio.SeqIO.parse(plasmidseqs, 'fasta')]
print("Read %d sequences from %s. These are:\n\t%s" % (len(plasmids), plasmidseqs, '\n\t'.join([seq.description for seq in plasmids])))

# Now get the plasmid coding sequences into plasmid_cdnas; this requires trimming the noncoding regions
plasmid_cdnas = {}
splicesites = {'M2':(26, 715), 'NS2':(30, 503)} # tuples give last nt of exon 1 & first of exon 2 in numbering starting with 1 at ATG
for cdna in cdnas:
    plasmidseq = [seq.seq for seq in plasmids if (cdna in seq.description) or (cdna in ['M1', 'NS1', 'M2', 'NS2'] and cdna[ : -1] in seq.description)]
    assert len(plasmidseq) == 1, "Failed to find a single plasmid sequence for %s" % cdna
    plasmidseq = plasmidseq[0]
    orfstart = str(plasmidseq).index('ATG')
    if cdna == 'NP': # starts with second ATG in RNA
        orf = plasmidseq[str(plasmidseq).index('ATG', orfstart + 1) : ]
    elif cdna not in ['M2', 'NS2']: # starts with first ATG in RNA
        orf = plasmidseq[orfstart : ]
    else: # things are bit more complicated for the spliced genes M2 and NS2
        (splice5, splice3) = splicesites[cdna]
        exon1 = plasmidseq[orfstart : orfstart + splice5]
        exon2 = plasmidseq[orfstart + splice3 - 1 : ]
        orf = exon1 + exon2
    prot = orf.translate(to_stop=True)
    orf = orf[ : 3 * len(prot)]
    plasmid_cdnas[cdna] = orf
    print("Found a reading frame of %s nucleotides (%d codons) for %s" % (len(orf), len(orf) // 3, cdna))

Read 8 sequences from ./sequences/plasmid_sequences_20150904.txt. These are:
	CA04-PB2 plasmid
	pYW472-PB1 plasmid
	CA04-PA plasmid
	CA04-HA plasmid
	CA04-NP plasmid
	pYW476-NA plasmid
	pYW477-M plasmid
	pYW478-NS plasmid
Found a reading frame of 2277 nucleotides (759 codons) for PB2
Found a reading frame of 2271 nucleotides (757 codons) for PB1
Found a reading frame of 2148 nucleotides (716 codons) for PA
Found a reading frame of 1698 nucleotides (566 codons) for HA
Found a reading frame of 1494 nucleotides (498 codons) for NP
Found a reading frame of 1407 nucleotides (469 codons) for NA
Found a reading frame of 756 nucleotides (252 codons) for M1
Found a reading frame of 291 nucleotides (97 codons) for M2
Found a reading frame of 657 nucleotides (219 codons) for NS1
Found a reading frame of 363 nucleotides (121 codons) for NS2


## Comparison of reference sequences to those in Genbank
We now compare the reference sequences from the plasmids to all sequences present for pandemic H1N1 that we have downloaded from the [Influenza Virus Resource](http://www.ncbi.nlm.nih.gov/genomes/FLU/FLU.html). Since the sequences should all be nearly identical, we don't perform true alignments -- we just make sure they are length-matched as there are no indels during the evolution of pdmH1N1. Below are the results of this analysis, which show that while not all of the **nucleotide** sequences are exact matches to *A/California/04/2009* database sequences, all of the **protein** sequences are matches to at least one *A/California/04/2009* sequence. Overall, this seems to indicate that the plasmid sequences are probably okay.

In [3]:
alignedseqs_by_cdna = dict([(cdna, []) for cdna in cdnas])
seqs_d_by_cdna = {}
for cdna in cdnas:
    seqs = seqs_by_cdna[cdna]
    refseq = plasmid_cdnas[cdna]
    refprot = str(refseq.translate())
    refseq = str(refseq)
    idents = []
    seqs_d = {refseq:[]}
    prots_d = {refseq:[]}
    for seq in seqs:
        if len(seq.seq) - 3 == len(refseq) and seq.seq[-3 : ] in ['TGA', 'TAA', 'TAG']:
            seq.seq = seq.seq[: -3] # trim stop codon
        if len(seq.seq) == len(refseq):
            ident = len([i for (i, nt_i) in enumerate(refseq) if nt_i == seq[i]]) / float(len(refseq))
            idents.append((ident, seq))
            alignedseqs_by_cdna[cdna].append(seq)
            prot = str(seq.seq.translate())
            seqstring = str(seq.seq)
            if seqstring in seqs_d:
                seqs_d[seqstring].append(seq.description)
            else:
                seqs_d[seqstring] = [seq.description]
            if prot in prots_d:
                prots_d[prot].append(seq.description)
            else:
                prots_d[prot] = [seq.description]
    idents.sort()
    seqs_d_by_cdna[cdna] = seqs_d
    print("\nFor %s, %d of %d sequences aligned gaplessly. The median and minimum nucleotide identities are %.3f and %.3f" % (cdna, len(alignedseqs_by_cdna[cdna]), len(seqs_by_cdna[cdna]), idents[len(idents) // 2][0], idents[-1][0]))
    print("The %s nucleotide sequence in the plasmid exactly matches %s known sequences." % (cdna, len(seqs_d[refseq])))
    print("The %s protein sequence in the plasmid exactly matches %s known sequences." % (cdna, len(prots_d[refprot])))
    if [head for head in seqs_d[refseq] if 'A/California/04/2009' in head]:
        print('The exact nucleotide matches for %s include A/California/04/2009 sequences:\n\t%s' % (cdna, '\n\t'.join([head for head in seqs_d[refseq] if 'A/California/04/2009' in head])))
    elif seqs_d[refseq]:
        print('The exact nucleotide matches for %s do not include A/California/04/2009 but do include %d other sequences including %s' % (cdna, len(seqs_d[refseq]), seqs_d[refseq][0]))
    else:
        closestseq = idents[-1][1]
        nt_muts = ['%s%d%s' % (nt_i, i + 1, closestseq.seq[i]) for (i, nt_i) in enumerate(refseq) if nt_i != closestseq.seq[i]]
        print("There are no exact nucleotide matches for %s. The closest sequence is:\n\t%s\n\tThis sequence has the following nucleotide mutations: %s" % (cdna, closestseq.description, ', '.join(nt_muts)))
    if [head for head in prots_d[refprot] if 'A/California/04/2009' in head]:
        print('The exact protein matches for %s include A/California/04/2009 sequences:\n\t%s' % (cdna, '\n\t'.join([head for head in prots_d[refprot] if 'A/California/04/2009' in head])))
    elif prots_d[refprot]:
        print('The exact protein matches for %s do not include A/California/04/2009 but do include %d other sequences including %s' % (cdna, len(prots_d[refprot]), prots_d[refprot][0]))
    else:
        print("There are NO exact protein matches for %s" % cdna)
 


For PB2, 5111 of 5114 sequences aligned gaplessly. The median and minimum nucleotide identities are 0.997 and 1.000
The PB2 nucleotide sequence in the plasmid exactly matches 16 known sequences.
The PB2 protein sequence in the plasmid exactly matches 1544 known sequences.
The exact nucleotide matches for PB2 include A/California/04/2009 sequences:
	cds:ACP41102 A/California/04/2009 2009/04/01 H1N1 PB2
	cds:ACP44157 A/California/04/2009 2009/04/01 H1N1 PB2
The exact protein matches for PB2 include A/California/04/2009 sequences:
	cds:ACP41102 A/California/04/2009 2009/04/01 H1N1 PB2
	cds:ACP44157 A/California/04/2009 2009/04/01 H1N1 PB2

For PB1, 5101 of 5125 sequences aligned gaplessly. The median and minimum nucleotide identities are 0.997 and 1.000
The PB1 nucleotide sequence in the plasmid exactly matches 0 known sequences.
The PB1 protein sequence in the plasmid exactly matches 1944 known sequences.
There are no exact nucleotide matches for PB1. The closest sequence is:
	cds:ADF27

## How closely do we expect the reference sequences to match other sequences?
The analysis above finds that some of the reference (plasmid) nucleotide sequences don't exactly match any other nucleotide sequences in the [Influenza Virus Resource]. Is this surprising? Should we expect a "valid" pdmH1N1 influenza sequence to exactly match other nucleotide sequences in the database?

Below we count how many times each of the sequences in the [Influenza Virus Resource] is found for each cDNA. The results below show that is not unusual for a sequence to have no other exact nucleotide matches in the database (this is true for all cDNAs except *NS2* and *M2*). Therefore, we probably shouldn't be concerned that some of the segments (specifically, *PB1*, *PA*, *NS1*) don't exactly match any other sequence in the [Influenza Virus Resource], since this is true for many sequences for these segments. This result simply indicates that the high genetic diversity of influenza means that most segments will tend to have a lot of genetic variants. We can draw confidence from the fact that the **protein** sequences for all of the reference (plasmid) sequences do match other protein sequences, and so probably don't contain harmful mutations.

[Influenza Virus Resource]: http://www.ncbi.nlm.nih.gov/genomes/FLU/FLU.html

In [14]:
for cdna in cdnas:
    nmatches = []
    for (seq, exactmatches) in seqs_d_by_cdna[cdna].items():
        nmatches += [len(exactmatches)] * len(exactmatches)
    runningcount = 0
    print("\nFor %s, here are the number of sequences with no more than the indicated number of exact nucleotide matches:" % cdna)
    for i in range(1, 3):
        runningcount += nmatches.count(i)
        print("\t%d sequences are found no more than %d times (this is %.3f of the total)" % (runningcount, i, runningcount / float(len(nmatches))))
    print("\t%d sequences are found more than %d times (this is %.3f of the total)" % (len(nmatches) - runningcount, i, 1.0 - runningcount / float(len(nmatches))))


For PB2, here are the number of sequences with no more than the indicated number of exact nucleotide matches:
	2630 sequences are found no more than 1 times (this is 0.515 of the total)
	3170 sequences are found no more than 2 times (this is 0.620 of the total)
	1941 sequences are found more than 2 times (this is 0.380 of the total)

For PB1, here are the number of sequences with no more than the indicated number of exact nucleotide matches:
	2572 sequences are found no more than 1 times (this is 0.504 of the total)
	3178 sequences are found no more than 2 times (this is 0.623 of the total)
	1923 sequences are found more than 2 times (this is 0.377 of the total)

For PA, here are the number of sequences with no more than the indicated number of exact nucleotide matches:
	2620 sequences are found no more than 1 times (this is 0.489 of the total)
	3250 sequences are found no more than 2 times (this is 0.607 of the total)
	2104 sequences are found more than 2 times (this is 0.393 of the 

## Design of synonymously barcoded variants
We will now design synonymously barcoded variants of all of the genes. We will choose barcodes that do not alter the protein sequence, which change at least two nearby nucleotides (to facilitate Illumina short-read sequencing), and which only make mutations to nucleotides that are quite common in the existing database set of pdmH1N1 sequences. The rationale is that if the synonymous mutations are common in naturally occurring sequences, then they are probably not deleterious.

We will attempt to put the mutations somewhat close the 3' end of the gene, in case we ever want to do polyA-based RNAseq. However, we will not put them too close as nucleotides in the noncoding termini are sometimes involved in genome packaging. For the *M* and *NS* segments, we will put the mutations in the *M2* and *NS2* coding regions (after their overlapping regions with *M1* and *NS1*).

Below is the design; at the end of the output are the created sequences and the names of the files to which they are written:

In [16]:
# we will take the first synonymous mutation that meets the following criteria:
maxdistance = 33 # look mutations to be within this many nucleotides of each other
minfreq = 0.02 # require mutations to be present in at least this fraction of all sequences
minstopdistance = 50 # require mutations to be at least this many nucleotides upstream of stop codon

# now find such mutations...
import math
recoded_cdnas = []
recoded_plasmids = []
for cdna in cdnas:
    if cdna in ['NS1', 'M1']:
        continue # we barcode in the NS2 / M2 regions instead
    refseq = plasmid_cdnas[cdna]
    alignedseqs = alignedseqs_by_cdna[cdna]
    mincounts = len(alignedseqs) * minfreq # only consider synonymous mutations that occur at least this many times
    syn_muts = [] # for each codon site with synonymous mutation that occurs >= mincounts time, is (site, most common syn mutation)
    for icodon in range(len(refseq) // 3):
        wtcodon = refseq[icodon * 3 : icodon * 3 + 3]
        codon_muts = {}
        for seq in alignedseqs:
            mutcodon = seq.seq[icodon * 3 : icodon * 3 + 3]
            if str(mutcodon) != str(wtcodon) and str(wtcodon.translate()) == str(mutcodon.translate()):
                if mutcodon in codon_muts:
                    codon_muts[mutcodon] += 1
                else:
                    codon_muts[mutcodon] = 1
        codon_muts = [(n, mutcodon) for (mutcodon, n) in codon_muts.items()]
        codon_muts.sort()
        if codon_muts and codon_muts[-1][0] >= mincounts:
            syn_muts.append((icodon + 1, wtcodon, codon_muts[-1][1]))
    print("\nFor %s, found %d synonymous mutations that occur in at least %.2f of sequences (this is %d sequences)" % (cdna, len(syn_muts), minfreq, mincounts))
    syn_muts.reverse()
    lastallowedcodon = int(math.ceil(len(refseq) - minstopdistance) / 3.)
    if cdna == 'NS2':
        # make sure site does not overlap with NS1 
        firstallowedcodon = (len(plasmid_cdnas['NS1']) - splicesites['NS2'][1] + splicesites['NS2'][0]) // 3 + 1
    elif cdna == 'M2':
        # make sure site does not overlap with M1 
        firstallowedcodon = (len(plasmid_cdnas['M1']) - splicesites['M2'][1] + splicesites['M2'][0]) // 3 + 1
    else:
        firstallowedcodon = 1
    for (i, (icodon, wtcodon, mutcodon)) in enumerate(syn_muts[ : -1]):
        if icodon > lastallowedcodon or icodon < firstallowedcodon:
            continue # too close to 3' end, might be involved in genome packaging
        (nexticodon, nextwtcodon, nextmutcodon) = syn_muts[i + 1]
        if (icodon - nexticodon) * 3 <= maxdistance: # sufficiently close
            break
    else:
        raise RuntimeError("Failed to find synonymous mutations for %s" % cdna)
    (mut1, mut2) = ('%s%d%s' % (nextwtcodon, nexticodon, nextmutcodon), '%s%d%s' % (wtcodon, icodon, mutcodon))
    print("We will make the following two codon mutations to %s: %s and %s" % (cdna, mut1, mut2))
    recoded = list(str(refseq))
    for (i, wt, mut) in [(nexticodon, nextwtcodon, nextmutcodon), (icodon, wtcodon, mutcodon)]:
        for (j, nt) in enumerate(wt):
            assert recoded[(i - 1) * 3 + j] == nt
            if mut[j] != nt:
                recoded[(i - 1) * 3 + j] = mut[j].lower()
    recoded = ''.join(recoded)
    assert len(refseq) == len(recoded)
    assert 2 == len([i for i in range(len(recoded)) if recoded[i] != str(refseq)[i]])
    newhead = '%s synonymously barcoded with codon mutations %s and %s' % (cdna, mut1, mut2)
    recoded_cdnas.append((newhead, recoded))
    if cdna in ['NS2', 'M2']:
        plasmid = [str(seq.seq) for seq in plasmids if cdna[ : -1] in seq.description]
        assert len(plasmid) == 1
        plasmid = plasmid[0]
        trim5 = splicesites[cdna][0] + 1
        i = plasmid.index(str(refseq)[trim5 : ])
        recodedplasmid = '%s%s%s' % (plasmid[ : i], recoded[trim5 : ], plasmid[i + len(recoded[trim5 : ]) : ])
        assert len(recodedplasmid) == len(plasmid), "%d vs %d" % (len(recodedplasmid), len(plasmid))
        assert 2 == len([j for j in range(len(plasmid)) if plasmid[j] != recodedplasmid[j]])
    else:
        plasmid = [str(seq.seq) for seq in plasmids if cdna in seq.description]
        assert len(plasmid) == 1
        plasmid = plasmid[0]
        i = plasmid.index(str(refseq))
        recodedplasmid = "%s%s%s" % (plasmid[: i], recoded, plasmid[i + len(recoded) : ])
        assert len(recodedplasmid) == len(plasmid)
        assert 2 == len([j for j in range(len(plasmid)) if plasmid[j] != recodedplasmid[j]])
    recoded_plasmids.append(('plasmid vRNA insert for %s' % head, recodedplasmid))

# write to files
recoded_cdnas_file = '%s/recoded_cDNAs.txt' % seqdir
recoded_plasmids_file = '%s/recoded_plasmids.txt' % seqdir
for (fname, seqtype, seqlist) in [(recoded_cdnas_file, 'cDNAs', recoded_cdnas), (recoded_plasmids_file, 'plasmids', recoded_plasmids)]:
    print("\nWriting the recoded %s to %s" % (seqtype, fname))
    with open(fname, 'w') as f:
        f.write('\n\n'.join(['>%s\n%s' % tup for tup in seqlist]))
        
print('\n\nHere are the recoded cDNAs in %s:\n' % recoded_cdnas_file)
!cat $recoded_cdnas_file

print('\n\nHere are the recoded plasmids in %s:\n' % recoded_plasmids_file)
!cat $recoded_plasmids_file
            


For PB2, found 38 synonymous mutations that occur in at least 0.02 of sequences (this is 102 sequences)
We will make the following two codon mutations to PB2: AAG721AAA and CAA728CAG

For PB1, found 39 synonymous mutations that occur in at least 0.02 of sequences (this is 102 sequences)
We will make the following two codon mutations to PB1: TTC730TTT and AAA737AAG

For PA, found 40 synonymous mutations that occur in at least 0.02 of sequences (this is 107 sequences)
We will make the following two codon mutations to PA: GAG597GAA and ACC608ACT

For HA, found 30 synonymous mutations that occur in at least 0.02 of sequences (this is 225 sequences)
We will make the following two codon mutations to HA: CTA470TTA and AAC479AAT

For NP, found 28 synonymous mutations that occur in at least 0.02 of sequences (this is 109 sequences)
We will make the following two codon mutations to NP: GAA454GAG and TTC458TTT

For NA, found 22 synonymous mutations that occur in at least 0.02 of sequences (this 